In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

#############################
# 0) Constants & Folders
#############################
STOCKS = ["AAPL", "AMGN", "TSLA", "JPM", "XOM"]
SECTORS = {
    "AAPL": "Tech",
    "AMGN": "Healthcare",
    "TSLA": "Consumer",
    "JPM":  "Financial",
    "XOM":  "Energy"
}
DATA_PATH = "../data"
RESULTS_PATH = "../results"
os.makedirs(DATA_PATH, exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)

##################################
# Helper: Cross-Impact Regressions
##################################
def run_crossimpact_regression(df_data, ret_cols, ofi_cols, shift=0):
    """
    For each 'log_ret_SYMBOL' in ret_cols, regress on all 'OFI_PCA_SYMBOL' (optionally shifted).
    Returns a dict of { stock_name: { 'r2': float, 'params': Series } }
    """
    out = {}
    for ret_col in ret_cols:
        # e.g. ret_col = "log_ret_AAPL"
        target_stock = ret_col.split("_")[-1]

        # Build design matrix of all OFI columns (shifted if needed).
        X = pd.DataFrame(index=df_data.index)
        for ofi_col in ofi_cols:
            if shift > 0:
                X[ofi_col] = df_data[ofi_col].shift(shift)
            else:
                X[ofi_col] = df_data[ofi_col]

        y = df_data[ret_col]

        # Drop NaNs
        tmp = pd.concat([X, y], axis=1).dropna()
        if tmp.shape[0] < 30:  # skip if insufficient data
            out[target_stock] = { "r2": None, "params": None }
            continue

        X_ols = sm.add_constant(tmp[ofi_cols])
        y_ols = tmp[ret_col]

        model = sm.OLS(y_ols, X_ols)
        res = model.fit()
        out[target_stock] = {
            "r2": res.rsquared,
            "params": res.params
        }
    return out

def build_coefficient_matrix(crossimpact_dict):
    """
    Creates an NxN DataFrame: row=stock i, col=stock j, value=Beta_i_j.
    (Beta_i_j is the coefficient for OFI_PCA_j in log_ret_i's regression.)
    """
    mat = pd.DataFrame(index=STOCKS, columns=STOCKS, dtype=float)
    for stock_i, info_i in crossimpact_dict.items():
        if info_i["params"] is not None:
            for stock_j in STOCKS:
                param_col = f"OFI_PCA_{stock_j}"
                if param_col in info_i["params"].index:
                    mat.loc[stock_i, stock_j] = info_i["params"][param_col]
    return mat

##################################
# 1) Load & Merge the 5 pickled DataFrames
##################################
df_dict = {}
for symbol in STOCKS:
    pkl_file = os.path.join(DATA_PATH, f"{symbol}_1m_aggregated.pkl")
    if not os.path.exists(pkl_file):
        print(f"Error: {pkl_file} not found. Run multi_stock_ofi.py first.")
        continue
    df_tmp = pd.read_pickle(pkl_file).sort_index()

    # rename to avoid collisions: OFI_PCA_{symbol}, log_ret_{symbol}, etc.
    df_tmp = df_tmp.rename(columns={
        "OFI_PCA":   f"OFI_PCA_{symbol}",
        "log_ret":   f"log_ret_{symbol}",
    })

    df_dict[symbol] = df_tmp

# Step: Combine them all by outer join on the time index
df_cross = None
for i, (symbol, df_tmp) in enumerate(df_dict.items()):
    if i == 0:
        df_cross = df_tmp
    else:
        # df_cross = df_cross.join(df_tmp, how="outer")
        df_cross = df_cross.join(df_tmp, how="outer", rsuffix=f"_{symbol}")

df_cross = df_cross.dropna(how="all")
print(f"Merged cross-impact DataFrame shape: {df_cross.shape}")

##################################
# 2) Analyze Cross-Impact
#    (a) Contemporaneous:  log_ret_i(t) ~ sum_j OFI_PCA_j(t)
#    (b) 1-min-lag:        log_ret_i(t) ~ sum_j OFI_PCA_j(t-1)
##################################
ret_cols = [f"log_ret_{s}"   for s in STOCKS]
ofi_cols = [f"OFI_PCA_{s}"   for s in STOCKS]

# (a) Contemporaneous
res_contemp = run_crossimpact_regression(df_cross, ret_cols, ofi_cols, shift=0)
# (b) 1-min-lag
res_lag1    = run_crossimpact_regression(df_cross, ret_cols, ofi_cols, shift=1)

# Summaries: Print R2
def print_summary(label, crossimpact_res):
    print(f"\n=== {label} Cross-Impact Summary ===")
    for stk, info in crossimpact_res.items():
        if info["r2"] is None:
            print(f"  {stk}: Not enough data.")
            continue
        print(f"  {stk}: R2 = {info['r2']:.4f}")
        print(f"    Params:\n{info['params']}\n")

print_summary("Contemporaneous", res_contemp)
print_summary("Lag-1",          res_lag1)

# Quick self-vs-cross comparison
def compare_self_vs_cross(label, crossimpact_res):
    print(f"\n--- Self vs. Cross in {label} Model ---")
    for stk, info in crossimpact_res.items():
        if info["r2"] is not None and info["params"] is not None:
            # e.g. "OFI_PCA_AAPL" vs. all others
            self_col   = f"OFI_PCA_{stk}"
            if self_col not in info["params"].index:
                continue
            self_val   = abs(info["params"][self_col])
            cross_vals = [
                abs(info["params"][c])
                for c in info["params"].index
                if c.startswith("OFI_PCA_") and c != self_col
            ]
            if cross_vals:
                max_cross = max(cross_vals)
                print(f"  {stk}: Self={self_val:.2e}, MaxCross={max_cross:.2e}")
            else:
                print(f"  {stk}: Only self param found (no cross).")

compare_self_vs_cross("Contemporaneous", res_contemp)
compare_self_vs_cross("Lag-1",          res_lag1)

##################################
# 3) Visualization & Reporting
##################################
# 3.1) Heatmaps
mat_contemp = build_coefficient_matrix(res_contemp)
mat_lag1    = build_coefficient_matrix(res_lag1)

def plot_heatmap(mat, title, out_png):
    plt.figure(figsize=(8,6))
    sns.heatmap(mat, annot=True, cmap="RdBu", center=0, fmt=".2e")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(out_png)
    plt.close()

plot_heatmap(mat_contemp, "Contemporaneous Cross-Impact (Coefficients)",
             os.path.join(RESULTS_PATH, "crossimpact_contemp_heatmap.png"))
plot_heatmap(mat_lag1,    "Lag-1 Cross-Impact (Coefficients)",
             os.path.join(RESULTS_PATH, "crossimpact_lag1_heatmap.png"))

# 3.2) Pairwise scatter among all "OFI_PCA_x" and "log_ret_x"
#     (If you want cross-pairs, that is NxN=25. We'll do a simpler approach.)
scatter_cols = []
for s in STOCKS:
    scatter_cols.extend([f"OFI_PCA_{s}", f"log_ret_{s}"])
df_scatter = df_cross[scatter_cols].dropna().sample(frac=0.1, random_state=42)
sns.pairplot(df_scatter, corner=True, plot_kws={"alpha":0.3})
plt.suptitle("OFI_PCA_x vs. log_ret_x (sample 10%)", y=1.02)
plt.savefig(os.path.join(RESULTS_PATH, "ofi_vs_ret_scattermatrix.png"))
plt.close()

print(f"Saved heatmaps & scatter plot to {RESULTS_PATH}/")

##################################
# 4) Bonus: Sector-Level Insights
#    - We create a correlation matrix by sector from the merged data
##################################
# We'll group OFI_PCA_x by sector, then average each sector's OFI_PCA,
# so we see how e.g. Tech vs. Healthcare might cross-influence.
df_sector = pd.DataFrame(index=df_cross.index)

# aggregate e.g. 'OFI_PCA_AAPL' under 'Tech'
for symbol in STOCKS:
    sector_name = SECTORS[symbol]
    col_ofi     = f"OFI_PCA_{symbol}"
    if col_ofi in df_cross.columns:
        if sector_name not in df_sector.columns:
            df_sector[sector_name] = df_cross[col_ofi]  # new col
        else:
            df_sector[sector_name] += df_cross[col_ofi] # sum them up

# Optionally average by # of symbols in the sector.
# We'll do a simple approach:
sector_counts = {}
for symbol in STOCKS:
    sector_name = SECTORS[symbol]
    sector_counts[sector_name] = sector_counts.get(sector_name, 0) + 1

for sec_col in df_sector.columns:
    df_sector[sec_col] = df_sector[sec_col] / sector_counts[sec_col]

df_sector = df_sector.dropna(how="all")  # remove empty rows

sector_corr = df_sector.corr()
plt.figure(figsize=(6,5))
sns.heatmap(sector_corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Sector-Level OFI_PCA Correlation")
plt.tight_layout()
plt.savefig(os.path.join(RESULTS_PATH, "sector_ofi_correlation.png"))
plt.close()
print("Saved sector-level correlation heatmap.")

##################################
# 4b) Bonus: Simple RandomForest example
##################################
# We'll pick, say, AAPL's 1-min-ahead returns as a target,
# and use all other stocks' OFI as features.
# This is purely for demonstration, to show "an advanced method".

aapl_ret_col = "log_ret_AAPL"
features_cols = [f"OFI_PCA_{s}" for s in STOCKS]  # all 5
# shift them by 1 minute, so we attempt to forecast next-minute AAPL return
df_rf = df_cross[[aapl_ret_col] + features_cols].copy()
for col in features_cols:
    df_rf[col+"_lag1"] = df_rf[col].shift(1)
df_rf.dropna(inplace=True)

X_rf = df_rf[[c+"_lag1" for c in features_cols]]
y_rf = df_rf[aapl_ret_col]

# Train/test split: last 20% as test
split_idx = int(len(df_rf)*0.8)
X_train, X_test = X_rf.iloc[:split_idx], X_rf.iloc[split_idx:]
y_train, y_test = y_rf.iloc[:split_idx], y_rf.iloc[split_idx:]

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
score = r2_score(y_test, y_pred)
print(f"\n[Bonus] RandomForest for 1-min-lag cross-OFI → AAPL's future returns, R2={score:.4f}")
print(f"Feature importances:\n{pd.Series(rf.feature_importances_, index=X_rf.columns)}")

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Merged cross-impact DataFrame shape: (5220, 65)

=== Contemporaneous Cross-Impact Summary ===
  AAPL: R2 = 0.0038
    Params:
const          -0.000010
OFI_PCA_AAPL    0.000005
OFI_PCA_AMGN   -0.000003
OFI_PCA_TSLA    0.000007
OFI_PCA_JPM    -0.000010
OFI_PCA_XOM    -0.000011
dtype: float64

  AMGN: R2 = 0.0037
    Params:
const           0.000009
OFI_PCA_AAPL   -0.000015
OFI_PCA_AMGN    0.000010
OFI_PCA_TSLA   -0.000010
OFI_PCA_JPM     0.000006
OFI_PCA_XOM    -0.000004
dtype: float64

  TSLA: R2 = 0.0062
    Params:
const          -1.052426e-05
OFI_PCA_AAPL   -3.548758e-05
OFI_PCA_AMGN   -1.021882e-05
OFI_PCA_TSLA   -1.145157e-05
OFI_PCA_JPM    -3.916741e-07
OFI_PCA_XOM    -1.487186e-05
dtype: float64

  JPM: R2 = 0.0036
    Params:
const           0.000002
OFI_PCA_AAPL   -0.000009
OFI_PCA_AMGN   -0.000002
OFI_PCA_TSLA   -0.000005
OFI_PCA_JPM     0.000008
OFI_PCA_XOM     0.000011
dtype: float64

  XOM: R2 = 0.0064
    Params:
const           0.000003
OFI_PCA_AAPL   -0.000015
OFI_PCA_AM

In [ ]:
# Code to add line plots OFI vs. returns for each symbol
import matplotlib.dates as mdates

for symbol in STOCKS:
    ofi_col = f"OFI_PCA_{symbol}"
    ret_col = f"log_ret_{symbol}"
    if ofi_col not in df_cross.columns or ret_col not in df_cross.columns:
        continue

    df_plot = df_cross[[ofi_col, ret_col]].dropna()
    if df_plot.empty:
        continue

    # Create figure
    fig, ax = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

    # 1) Plot OFI
    ax[0].plot(df_plot.index, df_plot[ofi_col], label=f"{ofi_col} (1-min sum)", color="steelblue")
    ax[0].axhline(y=0, color="black", linestyle="--", linewidth=0.8)
    ax[0].set_ylabel("OFI_PCA")
    ax[0].set_title(f"{symbol} OFI_PCA vs. 1-min Returns")
    ax[0].grid(True)
    ax[0].legend(loc="upper left")

    # 2) Plot log return
    ax[1].plot(df_plot.index, df_plot[ret_col], label=f"1-min log-return", color="orange")
    ax[1].axhline(y=0, color="black", linestyle="--", linewidth=0.8)
    ax[1].set_ylabel("Log Return")
    ax[1].grid(True)
    ax[1].legend(loc="upper left")

    # Format the x-axis for dates
    ax[1].xaxis.set_major_formatter(mdates.DateFormatter('%m-%d %H:%M'))

    plt.tight_layout()
    plt.savefig(f"{RESULTS_PATH}/{symbol}_ofi_vs_return_lineplot.png")
    plt.close()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
